# binary url encoding
Kunnen we codes handig van string naar bitmap omzetten en terug?

Yessur https://docs.python.org/2/library/binascii.html#binascii.hexlify

In [1]:
import pickle
import binascii

In [2]:
test_bools = [True, False, False, True]

In [3]:
bytearray(test_bools)

bytearray(b'\x01\x00\x00\x01')

In [4]:
binascii.hexlify(bytearray(test_bools))

b'01000001'

In [5]:
list(binascii.unhexlify(binascii.hexlify(bytearray(test_bools))))

[1, 0, 0, 1]

Hm, nee dat is niet echt de compacte representatie die ik in gedachten had... oh dit is beter: https://docs.python.org/3/library/base64.html

In [6]:
bin([True, False])

TypeError: 'list' object cannot be interpreted as an integer

In [7]:
def bool_list_to_binary_int(bool_list):
    return sum(int(v)*2**i for i,v in enumerate(bool_list[::-1]))

In [8]:
def bool_list_to_binary_repr(bool_list):
    return bin(bool_list_to_binary_int(bool_list))

In [9]:
bool_list_to_binary_repr(test_bools).encode('base64')

LookupError: 'base64' is not a text encoding; use codecs.encode() to handle arbitrary codecs

In [10]:
str(bool_list_to_binary_int(test_bools)).encode('base64')

LookupError: 'base64' is not a text encoding; use codecs.encode() to handle arbitrary codecs

In [11]:
bytes(bool_list_to_binary_repr(test_bools).encode('ascii'))

b'0b1001'

In [12]:
import base64

In [13]:
base64.urlsafe_b64encode(str(bool_list_to_binary_int(test_bools)))

TypeError: a bytes-like object is required, not 'str'

In [14]:
import BitVector

In [15]:
bv = BitVector.BitVector(bitlist=test_bools)

In [16]:
print(bv)

1001


In [21]:
import io
import codecs

tests met io weer weggehaald

In [22]:
bv = BitVector.BitVector(bitlist=test_bools * 2)

In [23]:
print(bv)

10011001


In [24]:
with open('testhy.txt', 'wb') as fh:
    bv.write_to_file(fh)

In [25]:
with open('testhy.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')

In [26]:
brap, proep

(b'\x99', b'mQ==\n')

In [27]:
bv_lang = BitVector.BitVector(bitlist=test_bools * 8)

In [28]:
print(bv_lang)

10011001100110011001100110011001


In [29]:
with open('testlang.txt', 'wb') as fh:
    bv_lang.write_to_file(fh)

In [30]:
with open('testlang.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')

In [31]:
brap, proep

(b'\x99\x99\x99\x99', b'mZmZmQ==\n')

In [32]:
bv_langer = BitVector.BitVector(bitlist=test_bools * 16)

In [33]:
with open('testlanger.txt', 'wb') as fh:
    bv_langer.write_to_file(fh)

In [34]:
with open('testlanger.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')

In [35]:
brap, proep

(b'\x99\x99\x99\x99\x99\x99\x99\x99', b'mZmZmZmZmZk=\n')

In [36]:
bv_langst = BitVector.BitVector(bitlist=test_bools * 32)

In [37]:
with open('testlangst.txt', 'wb') as fh:
    bv_langst.write_to_file(fh)
with open('testlangst.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')
brap, proep

(b'\x99\x99\x99\x99\x99\x99\x99\x99\x99\x99\x99\x99\x99\x99\x99\x99',
 b'mZmZmZmZmZmZmZmZmZmZmQ==\n')

In [38]:
bv_lung = BitVector.BitVector(bitlist=[1, 1, 1, 0] * 64 * 3)
with open('testlung.txt', 'wb') as fh:
    bv_lung.write_to_file(fh)
with open('testlung.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')
brap, proep

(b'\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee\xee',
 b'7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u\n7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u7u\n')

Die ='s aan het eind komen omdat base64 3x8=24 bits in 4 karakter opslaat, dus 6 bits per karakter. Als je dus geen factor 6 erin hebt dan krijg je een = aan het eind, wrs iets van de lege extra karakter ofzo.

Idd, zie hier, het is padding: https://stackoverflow.com/a/36571117/1199693

Kun je in een URL dus wel weglaten, alleen moet je ze dan weer toevoegen als je encoding string geen multiple van 4 is.

In [39]:
codecs.encode([True, False], 'base64')

TypeError: encoding with 'base64' codec failed (TypeError: expected bytes-like object, not list)

In [40]:
byte_file_like = io.BytesIO()

In [41]:
dinkie = [True, False, False, True, True, False, False, True]

In [42]:
byte_file_like.write(bytes(codecs.encode(str(dinkie), encoding='utf-8')))

52

In [43]:
byte_file_like.seek(0)

0

In [44]:
byte_file_like.read()

b'[True, False, False, True, True, False, False, True]'